In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm
import json
import concurrent.futures
import multiprocessing
import time

In [2]:
title_df = pd.read_csv('zippedData/title.basics.tsv.gz', delimiter='\t', error_bad_lines=False)

/Users/michaelroth/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
title_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950519 entries, 0 to 8950518
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 614.6+ MB


In [8]:
title_df = title_df.query('titleType == "movie"')
title_df = title_df.query('isAdult == 0')

title_df.startYear = title_df.startYear.apply(lambda x: np.nan if x == '\\N' else int(x))
title_df.dropna(inplace=True)
title_df.reset_index(drop=True, inplace=True)
title_df = title_df.query('startYear >= 2010 and startYear < 2022').sort_values('startYear').reset_index(drop=True)

title_df.drop(labels=['endYear', 'isAdult'], axis=1, inplace=True)

UndefinedVariableError: name 'isAdult' is not defined

In [10]:
title_df.runtimeMinutes = title_df.runtimeMinutes.apply(lambda x: np.nan if x == '\\N' else int(x))

In [19]:
title_df.query('runtimeMinutes > 60')

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt1816571,movie,Lyubov pod prikrytiem,Lyubov pod prikrytiem,2010,94.0,Drama
1,tt1082075,movie,Pusher,Pusher,2010,107.0,"Action,Crime,Drama"
2,tt1774264,movie,23,23,2010,104.0,"Action,Comedy"
3,tt1774269,movie,A Backyard Story,A Backyard Story,2010,70.0,Drama
4,tt1774281,movie,Aku atau Dia?,Aku atau Dia?,2010,97.0,"Comedy,Romance"
...,...,...,...,...,...,...,...
190357,tt14546488,movie,Lights Up J'ouvert,Lights Up J'ouvert,2021,98.0,Drama
190358,tt8438310,movie,Pinto e Shamy- Tears of star,Pinto e Shamy- Tears of star,2021,125.0,"Action,Adventure,Family"
190359,tt14546238,movie,Blue Box,Blue Box,2021,82.0,Documentary
190360,tt14546228,movie,MarginalizedAadhaar,MarginalizedAadhaar,2021,69.0,Documentary
